<a href="https://colab.research.google.com/github/FrederiKob/Kelly_Replication_Code/blob/main/Run_Rudimentary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import library_functions

ModuleNotFoundError: ignored